In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.downloader
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Sequential #, Dense, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-08-30 17:29:47.580766: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-30 17:29:47.629367: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 17:29:47.629399: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Early Experimenting in order to get to a BaseModel
#### with only minimal preprocessing

In [2]:
# Reading Data
data = pd.read_csv('../raw_data/small_dataset.csv')
data = data.drop(columns='Unnamed: 0')
data = data.rename(columns={'0': 'text'})
data.head()

,text,subtopic
0,2 2 0 2 g u A 6 2 ] E H . h p - o r t s a [ ...,Astrophysics
1,"Draft version August 29, 2022 Typeset using LA...",Astrophysics
2,Astronomy & Astrophysics manuscript no. 41891c...,Astrophysics
3,Astronomy & Astrophysics manuscript no. aa Aug...,Astrophysics
4,2 2 0 2 g u A 6 2 ] R S . h p - o r t s a [...,Astrophysics


In [3]:
# Tokenizing
data['modified text'] = data['text'].apply(word_tokenize)

In [4]:
# Stopwords Removal
stop_words = set(stopwords.words('english'))

data['modified text'] = data['modified text'].apply(lambda x: [word for word in x if not word in stop_words])
data['words per text'] = data['text'].apply(lambda x : len(x)) 
data['words per modified text'] = data['modified text'].apply(lambda x : len(x)) 
data

,text,subtopic,modified text,words per text,words per modified text
0,2 2 0 2 g u A 6 2 ] E H . h p - o r t s a [ ...,Astrophysics,"[2, 2, 0, 2, g, u, A, 6, 2, ], E, H, ., h, p, ...",58690,9146
1,"Draft version August 29, 2022 Typeset using LA...",Astrophysics,"[Draft, version, August, 29, ,, 2022, Typeset,...",55930,8499
2,Astronomy & Astrophysics manuscript no. 41891c...,Astrophysics,"[Astronomy, &, Astrophysics, manuscript, ., 41...",46117,6262
3,Astronomy & Astrophysics manuscript no. aa Aug...,Astrophysics,"[Astronomy, &, Astrophysics, manuscript, ., aa...",78395,12692
4,2 2 0 2 g u A 6 2 ] R S . h p - o r t s a [...,Astrophysics,"[2, 2, 0, 2, g, u, A, 6, 2, ], R, S, ., h, p, ...",61921,9731
...,...,...,...,...,...
2030,2 2 0 2 g u A 3 2 ] A F . h t a m [ 1 v 5...,Symplectic Geometry,"[2, 2, 0, 2, g, u, A, 3, 2, ], A, F, ., h, [, ...",70325,12866
2031,UNIVERSIDAD COMPLUTENSE DE MADRID FACULTAD DE...,Symplectic Geometry,"[UNIVERSIDAD, COMPLUTENSE, DE, MADRID, FACULTA...",79210,18640
2032,2 2 0 2 g u A 0 2 ] G S . h t a m [ 1 v 4...,Symplectic Geometry,"[2, 2, 0, 2, g, u, A, 0, 2, ], G, S, ., h, [, ...",46697,9457
2033,2 2 0 2 g u A 6 1 ] G S . h t a m [ 1 v 4...,Symplectic Geometry,"[2, 2, 0, 2, g, u, A, 6, 1, ], G, S, ., h, [, ...",26625,5040


In [5]:
data.columns

Index(['text', 'subtopic', 'modified text', 'words per text',
       'words per modified text'],
      dtype='object')

In [6]:
# Add Padding
data['modified text padded'] = pad_sequences(data['modified text'], dtype='float32', padding='post', value=0)

ValueError: could not convert string to float: 'g'

In [ ]:
# ToDo: Target Encoding, but turning subtopics into topics first
def target_encoding():
    

data['subtopic'].map_apply()

In [ ]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(data['modified text'], data['subtopic'], test_size=0.3)

In [8]:
# Downloading a pre-trained model, based on a 50 space vector representation from wikidata
model_wiki = gensim.downloader.load('glove-wiki-gigaword-50')

In [12]:
model_wiki['saturn'].shape

(50,)

In [ ]:
glove_file = datapath('~/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz')
tmp_file = get_tmpfile("test_word2vec.txt")
#tmp_file = data['modified text']

_ = glove2word2vec(glove_file, tmp_file)

glove_vectors = KeyedVectors.load_word2vec_format(tmp_file)

In [ ]:
model_wiki.most_similar("glass")

### Neural network architechture with three components:

    An embedding layer that generates word embedding, and the parameters are shared across words.
    A hidden layer of one or more layers, which introduces non-linearity to the embeddings.
    A softmax function that produces probability distribution over all the words in the vocabulary. 



In [ ]:
embedding_size = 50 # same as in the pretrained model

model = Sequential()

model.add(layers.LSTM(20))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])